In [2]:
!pip install -q transformers torch sentencepiece


In [3]:
from transformers import pipeline


In [4]:
classifier = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cuda:0


In [22]:
urgent_keywords = [
    "down", "not working", "failure", "failed",
    "urgent", "asap", "immediately",
    "outage", "crash", "broken", "stopped",
    "unacceptable", "frustrated"
]


In [25]:
def classify_ticket(ticket):
    result = classifier(ticket)[0]

    label = result["label"]     # LABEL_0, LABEL_1, LABEL_2
    score = result["score"]

    # RoBERTa sentiment mapping
    is_negative = label == "LABEL_0"

    # Urgency keywords
    urgent_keywords = [
        "down", "not working", "failure", "failed",
        "urgent", "asap", "immediately",
        "outage", "crash", "broken", "stopped",
        "unacceptable", "frustrated"
    ]

    is_urgent = any(word in ticket.lower() for word in urgent_keywords)

    # FINAL LOGIC
    if is_negative or is_urgent:
        category = "Angry/Urgent"
        priority = "P1 - Immediate Attention"
    else:
        category = "Neutral"
        priority = "P3 - Normal Priority"

    return {
        "ticket": ticket,
        "model_label": label,
        "confidence": round(score * 100, 2),
        "category": category,
        "priority": priority
    }


In [26]:
test_tickets = [
    "My server is down and this is unacceptable!",
    "Please reset my password",
    "I want my ticket Urgent"
]

for t in test_tickets:
    print(classify_ticket(t))


{'ticket': 'My server is down and this is unacceptable!', 'model_label': 'LABEL_0', 'confidence': 97.78, 'category': 'Angry/Urgent', 'priority': 'P1 - Immediate Attention'}
{'ticket': 'Please reset my password', 'model_label': 'LABEL_1', 'confidence': 65.15, 'category': 'Neutral', 'priority': 'P3 - Normal Priority'}
{'ticket': 'I want my ticket Urgent', 'model_label': 'LABEL_1', 'confidence': 68.96, 'category': 'Angry/Urgent', 'priority': 'P1 - Immediate Attention'}
